In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt

import re, os
import unicodedata
import json

import nltk
import nltk.sentiment
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from time import strftime

from wordcloud import WordCloud

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

import acquire

import spacy
pd.set_option('display.max_colwidth', None)



### Acquire data

We will be using the 1000 labeled notes in `test.csv`

In [8]:
import prepare_jag

In [11]:
df = prepare_jag.prep_train()

Test, notes, and features loaded.
Merged dataframes
Renamed 'pn_history' column to 'original'
Added a basic clean column lowercaseing and removing special characters
Added stemmed column with tokenized words and stopwords removed
Added lemmatized column with lemmatized words and stopwords removed
Data preparation complete


In [16]:
df.head(1)


,id,case_num_x,pn_num,feature_num,annotation,location,case_num_y,original,clean,stemmed,lemmatized
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],0,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with ""thyroid disease,"" dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms",hpi 17yo presents palpitations patient reports 34 months intermittent episodes heart beatingpounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 13 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes visionhearing abdominal paun changes bowel urinary habits pmhx none rx uses friends adderrall fhx mom thyroid disease dad recent heart attcak none immunizations date shx freshmen college endorses 34 drinks 3 nights week weekends denies tabacco endorses trying marijuana sexually active girlfriend x 1 year uses condoms,hpi 17yo present palpit patient report 34 month intermitt episod heart beatingpound chest 2 day ago dure soccer game episod thi time chest pressur felt go pass lose concious note patient endors abus adderal primarili studi 13 time per week befor recent soccer game took adderral night befor morn game deni short breath diaphoresi fever chill headach fatigu chang sleep chang visionhear abdomin paun chang bowel urinari habit pmhx none rx use friend adderral fhx mom thyroid diseas dad recent heart attcak none immun date shx freshmen colleg endors 34 drink 3 night week weekend deni tabacco endors tri marijuana sexual activ girlfriend x 1 year use condom,hpi 17yo present palpitation patient report 34 month intermittent episode heart beatingpounding chest 2 day ago soccer game episode time chest pressure felt going pas lose conciousness note patient endorses abusing adderall primarily study 13 time per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fever chill headache fatigue change sleep change visionhearing abdominal paun change bowel urinary habit pmhx none rx us friend adderrall fhx mom thyroid disease dad recent heart attcak none immunization date shx freshman college endorses 34 drink 3 night week weekend denies tabacco endorses trying marijuana sexually active girlfriend x 1 year us condom


---

## Modeling

### Split Data

- Split in to train / test (700/300)


### X/y split

- Let x be the TFID vectorization of the lemmatized word list
- y = case number

In [47]:
df.head(1)

,id,case_num_x,pn_num,feature_num,annotation,location,case_num_y,original,clean,stemmed,lemmatized
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],0,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with ""thyroid disease,"" dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms",hpi 17yo presents palpitations patient reports 34 months intermittent episodes heart beatingpounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 13 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes visionhearing abdominal paun changes bowel urinary habits pmhx none rx uses friends adderrall fhx mom thyroid disease dad recent heart attcak none immunizations date shx freshmen college endorses 34 drinks 3 nights week weekends denies tabacco endorses trying marijuana sexually active girlfriend x 1 year uses condoms,hpi 17yo present palpit patient report 34 month intermitt episod heart beatingpound chest 2 day ago dure soccer game episod thi time chest pressur felt go pass lose concious note patient endors abus adderal primarili studi 13 time per week befor recent soccer game took adderral night befor morn game deni short breath diaphoresi fever chill headach fatigu chang sleep chang visionhear abdomin paun chang bowel urinari habit pmhx none rx use friend adderral fhx mom thyroid diseas dad recent heart attcak none immun date shx freshmen colleg endors 34 drink 3 night week weekend deni tabacco endors tri marijuana sexual activ girlfriend x 1 year use condom,hpi 17yo present palpitation patient report 34 month intermittent episode heart beatingpounding chest 2 day ago soccer game episode time chest pressure felt going pas lose conciousness note patient endorses abusing adderall primarily study 13 time per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fever chill headache fatigue change sleep change visionhearing abdominal paun change bowel urinary habit pmhx none rx us friend adderrall fhx mom thyroid disease dad recent heart attcak none immunization date shx freshman college endorses 34 drink 3 night week weekend denies tabacco endorses trying marijuana sexually active girlfriend x 1 year us condom


In [48]:
df.dtypes

id             object
case_num_x      int64
pn_num          int64
feature_num     int64
annotation     object
location       object
case_num_y      int64
original       object
clean          object
stemmed        object
lemmatized     object
dtype: object

HAving issues 

In [49]:
#df.case_num_x=df.case_num_x.astype('object')

In [50]:
df.dtypes

id             object
case_num_x      int64
pn_num          int64
feature_num     int64
annotation     object
location       object
case_num_y      int64
original       object
clean          object
stemmed        object
lemmatized     object
dtype: object

Running into issue with number that isnt really a number for a target rename

In [1]:
############### Column Lambdas###############


def categorise(row):
    if row['case_num_x'] == 0:
        return 'case0'
    elif row['case_num_x'] == 1:
        return 'case1'
    elif row['case_num_x'] == 2:
        return 'case2'
    elif row['case_num_x'] == 3:
        return 'case3'
    elif row['case_num_x'] == 4:
        return 'case3'
    elif row['case_num_x'] == 5:
        return 'case5'
    elif row['case_num_x'] == 6:
        return 'case6'
    elif row['case_num_x'] == 7:
        return 'case7'
    elif row['case_num_x'] == 8:
        return 'case8'
    return 'case9'

In [2]:
    df['target'] = df.apply(lambda row: categorise(row), axis=1)


NameError: name 'df' is not defined

In [53]:
df.head(1)

,id,case_num_x,pn_num,feature_num,annotation,location,case_num_y,original,clean,stemmed,lemmatized,target
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],0,"HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of ""heart beating/pounding out of my chest."" 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with ""thyroid disease,"" dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms",hpi 17yo presents palpitations patient reports 34 months intermittent episodes heart beatingpounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 13 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes visionhearing abdominal paun changes bowel urinary habits pmhx none rx uses friends adderrall fhx mom thyroid disease dad recent heart attcak none immunizations date shx freshmen college endorses 34 drinks 3 nights week weekends denies tabacco endorses trying marijuana sexually active girlfriend x 1 year uses condoms,hpi 17yo present palpit patient report 34 month intermitt episod heart beatingpound chest 2 day ago dure soccer game episod thi time chest pressur felt go pass lose concious note patient endors abus adderal primarili studi 13 time per week befor recent soccer game took adderral night befor morn game deni short breath diaphoresi fever chill headach fatigu chang sleep chang visionhear abdomin paun chang bowel urinari habit pmhx none rx use friend adderral fhx mom thyroid diseas dad recent heart attcak none immun date shx freshmen colleg endors 34 drink 3 night week weekend deni tabacco endors tri marijuana sexual activ girlfriend x 1 year use condom,hpi 17yo present palpitation patient report 34 month intermittent episode heart beatingpounding chest 2 day ago soccer game episode time chest pressure felt going pas lose conciousness note patient endorses abusing adderall primarily study 13 time per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fever chill headache fatigue change sleep change visionhearing abdominal paun change bowel urinary habit pmhx none rx us friend adderrall fhx mom thyroid disease dad recent heart attcak none immunization date shx freshman college endorses 34 drink 3 night week weekend denies tabacco endorses trying marijuana sexually active girlfriend x 1 year us condom,case0


In [54]:
# Split X Y
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df.lemmatized)
y = df.target

In [55]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2, random_state=123)

## Baseline

In [56]:
df.target.value_counts()

case3    2600
case5    1800
case8    1800
case2    1700
case9    1700
case0    1300
case1    1300
case6    1200
case7     900
Name: target, dtype: int64

Can set baseline to be case 3

In [78]:
# baseline
df[df['target'] == 'case3'].target.value_counts()/sum(df.target.value_counts())

case3    0.181818
Name: target, dtype: float64

Baseline is 18.6%

In [76]:
#Create evaluation dataframe
train = pd.DataFrame(dict(actual=y_train))
train['baseline']='case3'


In [77]:
train.head()

,actual,baseline
1243,case0,case3
13593,case9,case3
5786,case3,case3
7362,case5,case3
6791,case3,case3


In [60]:
train.dtypes

actual      object
baseline    object
dtype: object

In [79]:
# Calculate baseline model performance
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.baseline)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.actual, train.baseline))
print('---')
print(classification_report(train.actual, train.baseline))


Accuracy: 18.18%
---
Confusion Matrix
baseline  case3
actual         
case0      1040
case1      1040
case2      1360
case3      2080
case5      1440
case6       960
case7       720
case8      1440
case9      1360
---
              precision    recall  f1-score   support

       case0       0.00      0.00      0.00      1040
       case1       0.00      0.00      0.00      1040
       case2       0.00      0.00      0.00      1360
       case3       0.18      1.00      0.31      2080
       case5       0.00      0.00      0.00      1440
       case6       0.00      0.00      0.00       960
       case7       0.00      0.00      0.00       720
       case8       0.00      0.00      0.00      1440
       case9       0.00      0.00      0.00      1360

    accuracy                           0.18     11440
   macro avg       0.02      0.11      0.03     11440
weighted avg       0.03      0.18      0.06     11440



/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


---

## Create Models

### Logistic Regression

In [80]:
# Make and fit the object
lm = LogisticRegression().fit(X_train, y_train)
# Use it to make predictions
train['lm_predicted'] = lm.predict(X_train)
# Asssess accuracy
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.lm_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.lm_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.lm_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual        case0  case1  case2  case3  case5  case6  case7  case8  case9
lm_predicted                                                               
case0          1040      0      0      0      0      0      0      0      0
case1             0   1040      0      0      0      0      0      0      0
case2             0      0   1360      0      0      0      0      0      0
case3             0      0      0   2080      0      0      0      0      0
case5             0      0      0      0   1440      0      0      0      0
case6             0      0      0      0      0    960      0      0      0
case7             0      0      0      0      0      0    720      0      0
case8             0      0      0      0      0      0      0   1440      0
case9             0      0      0      0      0      0      0      0   1360
---
              precision    recall  f1-score   support

       case0       1.00      1.00      1.00      1040
       case1

In [81]:
cross_val_score(lm, X_train, y_train, cv = 5)


array([1., 1., 1., 1., 1.])

### Decision Tree

In [82]:
# Make and fit the object
dtc = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train)
# Use the object
train['dt_predicted'] = dtc.predict(X_train)
# Determine performance
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.dt_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.dt_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.dt_predicted))

Accuracy: 42.60%
---
Confusion Matrix
actual        case0  case1  case2  case3  case5  case6  case7  case8  case9
dt_predicted                                                               
case2            11     11   1295      0      0      0     56      0      0
case3          1029   1029     65   1913   1440    950    395     44   1360
case7             0      0      0      0      0      0    269      0      0
case8             0      0      0    167      0     10      0   1396      0
---
              precision    recall  f1-score   support

       case0       0.00      0.00      0.00      1040
       case1       0.00      0.00      0.00      1040
       case2       0.94      0.95      0.95      1360
       case3       0.23      0.92      0.37      2080
       case5       0.00      0.00      0.00      1440
       case6       0.00      0.00      0.00       960
       case7       1.00      0.37      0.54       720
       case8       0.89      0.97      0.93      1440
       case9   

/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
cross_val_score(dtc, X_train, y_train, cv = 5)


array([0.42482517, 0.42701049, 0.42701049, 0.42657343, 0.42438811])

### Random Forest

In [66]:
# Make and fit object
rf = RandomForestClassifier(bootstrap = True, 
                            class_weight = None, 
                            criterion = 'gini',
                            min_samples_leaf = 3,
                            n_estimators = 100,
                            max_depth = 8, 
                            random_state = 123).fit(X_train, y_train)
# Use it to make predictions
train['rf_predicted'] = rf.predict(X_train)
# Assess performance
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.rf_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.rf_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.rf_predicted))

Accuracy: 99.87%
---
Confusion Matrix
actual        case0  case1  case2  case3  case5  case6  case7  case8  case9
rf_predicted                                                               
case0          1040      0      0      0      0      0      0      0      0
case1             0   1040      0      0      0      0      0      0      0
case2             0      0   1360      0      0      0      0      0      0
case3             0      0      0   2080      0      0     15      0      0
case5             0      0      0      0   1440      0      0      0      0
case6             0      0      0      0      0    960      0      0      0
case7             0      0      0      0      0      0    705      0      0
case8             0      0      0      0      0      0      0   1440      0
case9             0      0      0      0      0      0      0      0   1360
---
              precision    recall  f1-score   support

       case0       1.00      1.00      1.00      1040
       case1 

In [83]:
cross_val_score(rf, X_train, y_train, cv = 5)


array([0.99868881, 0.99694056, 0.99956294, 1.        , 0.99694056])

### KNN

In [84]:
# Make and fit the object
knn = KNeighborsClassifier(n_neighbors = 4).fit(X_train, y_train)
# Use the object 
train['knn_predicted'] = knn.predict(X_train)
# Evaluate performance
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.knn_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.knn_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.knn_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual         case0  case1  case2  case3  case5  case6  case7  case8  case9
knn_predicted                                                               
case0           1040      0      0      0      0      0      0      0      0
case1              0   1040      0      0      0      0      0      0      0
case2              0      0   1360      0      0      0      0      0      0
case3              0      0      0   2080      0      0      0      0      0
case5              0      0      0      0   1440      0      0      0      0
case6              0      0      0      0      0    960      0      0      0
case7              0      0      0      0      0      0    720      0      0
case8              0      0      0      0      0      0      0   1440      0
case9              0      0      0      0      0      0      0      0   1360
---
              precision    recall  f1-score   support

       case0       1.00      1.00      1.00      1040
 

In [69]:
# 5-fold Cross Validation
cross_val_score(knn, X_train, y_train, cv = 5)


array([1., 1., 1., 1., 1.])

### Support Vector Classifier

In [70]:
# Make and fit the object
svc = LinearSVC(random_state=0).fit(X_train, y_train)
# Use the object
train['svc_predicted'] = svc.predict(X_train)
# Evaluate model
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.svc_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.svc_predicted, train.actual))
print('---')
print(classification_report(train.actual, train.svc_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual         case0  case1  case2  case3  case5  case6  case7  case8  case9
svc_predicted                                                               
case0           1040      0      0      0      0      0      0      0      0
case1              0   1040      0      0      0      0      0      0      0
case2              0      0   1360      0      0      0      0      0      0
case3              0      0      0   2080      0      0      0      0      0
case5              0      0      0      0   1440      0      0      0      0
case6              0      0      0      0      0    960      0      0      0
case7              0      0      0      0      0      0    720      0      0
case8              0      0      0      0      0      0      0   1440      0
case9              0      0      0      0      0      0      0      0   1360
---
              precision    recall  f1-score   support

       case0       1.00      1.00      1.00      1040
 

In [71]:
# 5-fold Cross Validation
cross_val_score(svc, X_train, y_train, cv = 5)


array([1., 1., 1., 1., 1.])

---

## Test

Test Knn model

In [85]:
# Create testing dataframe
test = pd.DataFrame(dict(actual=y_test))
test['knn_predicted'] = knn.predict(X_test)
# Evaluate model
print('Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.knn_predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(test.knn_predicted, test.actual))
print('---')
print(classification_report(test.actual, test.knn_predicted))

Accuracy: 100.00%
---
Confusion Matrix
actual         case0  case1  case2  case3  case5  case6  case7  case8  case9
knn_predicted                                                               
case0            260      0      0      0      0      0      0      0      0
case1              0    260      0      0      0      0      0      0      0
case2              0      0    340      0      0      0      0      0      0
case3              0      0      0    520      0      0      0      0      0
case5              0      0      0      0    360      0      0      0      0
case6              0      0      0      0      0    240      0      0      0
case7              0      0      0      0      0      0    180      0      0
case8              0      0      0      0      0      0      0    360      0
case9              0      0      0      0      0      0      0      0    340
---
              precision    recall  f1-score   support

       case0       1.00      1.00      1.00       260
 

In [87]:
# Calculate percent improvement from baseline accuracy (47%)
pct_improvement = round(((100-18.18)/(18.18)*100), 2)
print(f'Our 100% accuracy represent a {pct_improvement}% improvement from baseline')

Our 100% accuracy represent a 450.06% improvement from baseline
